<a href="https://colab.research.google.com/github/Hemishahuja/eiffel_hello_world/blob/main/Shor15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See how to use Shor's algorithm to factor 15 here:    <b><a href="https://portal.quantumrings.com/doc/Shors.html">Shor15</a></b>

<i><b>Source code to factorize 15</b></i>

Note: Be sure to use your API token and your account name.

In [2]:
pip install QuantumRingsLib

Step 1. Import the required modules and obtain the backend

In [3]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math


provider = QuantumRingsProvider(
    token='rings-128.1wL7ihhPLHhG4qxog113VrcdBrXirlaM',
    name='haha1007@my.yorku.ca'
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 50000

provider.active_account()

{'name': 'haha1007@my.yorku.ca',
 'token': 'rings-128.1wL7ihhPLHhG4qxog113VrcdBrXirlaM',
 'max_qubits': '128'}

Step 2. Define the core methods

In [ ]:
############################################
# FINAL SHOR'S ALGORITHM CODE
############################################

import QuantumRingsLib
from QuantumRingsLib import (
    QuantumRegister,
    AncillaRegister,
    ClassicalRegister,
    QuantumCircuit,
    QuantumRingsProvider,
    job_monitor,
    JobStatus
)
from matplotlib import pyplot as plt
import numpy as np
import math

############################################
# 1. Configure Quantum Rings Backend
############################################
# Replace with your actual token and account name:
provider = QuantumRingsProvider(
    token='rings-128.1wL7ihhPLHhG4qxog113VrcdBrXirlaM',
    name='haha1007@my.yorku.ca'
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

# Check your account (optional)
provider.active_account()

############################################
# 2. Utility Functions
############################################

def iqft_circuit(qc, qubits, n):
    """
    Inverse QFT on 'n' qubits within the register 'qubits'.
    """
    # Swap qubit i with qubit n-i-1
    for i in range(n // 2):
        qc.swap(qubits[i], qubits[n - i - 1])

    # Apply the controlled phase and H gates
    for i in range(n):
        for j in range(i):
            qc.cu1(-math.pi / float(2**(i - j)), qubits[j], qubits[i])
        qc.h(qubits[i])

    qc.barrier()
    return qc


def plot_histogram(counts, title=""):
    """
    Plot the histogram of measurement outcomes.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    xvals = list(counts.keys())
    yvals = list(counts.values())
    plt.bar(xvals, yvals)
    plt.xlabel("Measurement Outcome")
    plt.ylabel("Counts")
    plt.title(title)
    plt.xticks(rotation=90)
    plt.show()


############################################
# 3. Modular Exponentiation Subcircuit (Placeholder)
############################################
def c_mult_mod_N(qc, ctl, x_reg, aux_reg, a, N, n):
    """
    Controlled modular multiplication by 'a' mod N.
    (Placeholder: implement logic in a real solution.)
    """
    # In a full implementation, do:
    #   if ctl == 1: x_reg = (x_reg * a) mod N
    # using quantum adders & modular reduction.
    pass

def controlled_mod_exp(qc, ctl_reg, x_reg, aux_reg, a, N, n):
    """
    Build the circuit for a^x mod N, controlled by each qubit in ctl_reg.
    """
    current_power = 1
    for i in range(len(ctl_reg)):
        # a^(2^i) mod N
        current_power = pow(a, 2**i, N)
        c_mult_mod_N(qc, ctl_reg[i], x_reg, aux_reg, current_power, N, n)


############################################
# 4. Period Estimation
############################################

def estimate_period(measured_value, Q, a, N):
    """
    Estimate the period r from the measured_value / Q using continued fractions.
    """
    from fractions import Fraction

    frac = Fraction(measured_value, Q).limit_denominator(N)
    candidate_r = frac.denominator

    # Check around candidate_r in a small range
    for k in range(-2, 3):
        test_r = candidate_r + k
        if test_r <= 0:
            continue
        if pow(a, test_r, N) == 1:
            return test_r
    return None


############################################
# 5. Shor's Algorithm for Large N
############################################
def shors_factorization(N, a=None, shots=1024):
    """
    Attempt Shor's algorithm once to factor N with base 'a' (optional).
    Returns (factor1, factor2) or (None, None) if unsuccessful.
    """
    from math import gcd
    import random

    if N < 2:
        raise ValueError("N must be >= 2")

    # 1) Pick a random 'a' if not given
    if not a:
        for _ in range(1000):
            candidate = random.randint(2, N-2)
            if gcd(candidate, N) == 1:
                a = candidate
                break
    if not a:
        print("Failed to find 'a' coprime with N.")
        return None, None

    # 2) Determine qubit counts
    n = math.ceil(math.log2(N))
    counting_qubits = 2 * n
    ancilla_qubits = n

    print(f"Factoring N={N} (roughly {n} bits) with base a={a}")
    print(f"Total qubits = {counting_qubits} (counting) + {n} (x_reg) + {ancilla_qubits} (ancillas)")

    # 3) Build quantum registers
    ctl_reg = QuantumRegister(counting_qubits, "ctl")
    x_reg   = QuantumRegister(n, "x")
    aux_reg = QuantumRegister(ancilla_qubits, "aux")
    c_reg   = ClassicalRegister(counting_qubits, "c")

    qc = QuantumCircuit(ctl_reg, x_reg, aux_reg, c_reg)

    # 4) Initialize x_reg to |1>
    qc.x(x_reg[n - 1])

    # 5) Put control register in superposition
    for i in range(counting_qubits):
        qc.h(ctl_reg[i])
    qc.barrier()

    # 6) Controlled modular exponentiation
    controlled_mod_exp(qc, ctl_reg, x_reg, aux_reg, a, N, n)
    qc.barrier()

    # 7) Inverse QFT on control register
    iqft_circuit(qc, ctl_reg, counting_qubits)

    # 8) Measure
    for i in range(counting_qubits):
        qc.measure(ctl_reg[i], c_reg[i])

    # 9) Execute
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    # 10) Classical post-processing
    from math import gcd
    measurement_outcome = max(counts, key=counts.get)  # highest freq
    measured_int = int(measurement_outcome, 2)
    Q = 2 ** counting_qubits

    r = estimate_period(measured_int, Q, a, N)
    if r is None or r == 0:
        print("Failed to find a valid period. Try a different 'a' or more shots.")
        return None, None

    factor1 = gcd(pow(a, r // 2, N) - 1, N)
    factor2 = gcd(pow(a, r // 2, N) + 1, N)

    if factor1 == 1 or factor1 == N:
        print("Period found, but factors were trivial. Try again.")
        return None, None

    print(f"Factors of {N} are: {factor1} and {factor2}")
    return factor1, factor2


def shors_factorization_with_retries(N, max_attempts=10, shots=1024):
    """
    Run Shor's algorithm multiple times (up to max_attempts),
    picking a new random 'a' each time, until it finds valid factors.
    """
    from math import gcd
    attempt = 0

    while attempt < max_attempts:
        attempt += 1
        print(f"\nAttempt {attempt} of {max_attempts}...")
        factor1, factor2 = shors_factorization(N, a=None, shots=shots)
        if factor1 and factor2:
            print(f"SUCCESS: Found factors after {attempt} attempts.")
            return factor1, factor2
        else:
            print("No non-trivial factors found in this attempt. Trying again...")

    print(f"\nFailed to factor {N} in {max_attempts} attempts.")
    return None, None


############################################
# 6. Example Usage
############################################
if __name__ == "__main__":
    #
    # PART A: Factor a ~17-bit integer with multiple attempts
    #
    N_large = 1019 * 1039  # 105741
    f1, f2 = shors_factorization_with_retries(N_large, max_attempts=5, shots=1024)
    if f1 and f2:
        print(f"\nFINAL RESULT for N={N_large}: {f1} x {f2}")
    else:
        print(f"\nNo luck factoring N={N_large} in 5 attempts.")

    #
    # PART B: Demonstrate factoring 15 with a simpler circuit
    #
    q2 = QuantumRegister(7, 'q')
    c2 = ClassicalRegister(4, 'c')
    qc2 = QuantumCircuit(q2, c2)

    # 1) Initialize: H on qubits 0,1,2; X on qubit 6
    qc2.h(q2[0])
    qc2.h(q2[1])
    qc2.h(q2[2])
    qc2.x(q2[6])
    qc2.barrier()

    # 2) Example modular exponentiation for 7^x mod 15
    qc2.cx(q2[2], q2[4])
    qc2.cx(q2[2], q2[5])
    qc2.cx(q2[6], q2[4])
    qc2.ccx(q2[1], q2[5], q2[3])
    qc2.cx(q2[3], q2[5])
    qc2.ccx(q2[1], q2[4], q2[6])
    qc2.cx(q2[6], q2[4])
    qc2.barrier()

    # 3) Inverse QFT on qubits [0..2]
    iqft_circuit(qc2, q2, 3)

    # 4) Measure qubits 0,1,2 -> c2[0,1,2]
    qc2.measure(q2[0], c2[0])
    qc2.measure(q2[1], c2[1])
    qc2.measure(q2[2], c2[2])

    # Execute
    job2 = backend.run(qc2, shots=shots)
    job_monitor(job2)
    result2 = job2.result()
    counts2 = result2.get_counts()

    # Show results
    plot_histogram(counts2, title="Measurement Outcomes for factoring 15")
    print("\nCounts for factoring 15:", counts2)

    # (Optional) Cleanup
    del qc2, q2, c2, result2, job2


Step 3. Perform the algorithm

The circuit to factor 15 shown above.

A plot of the execution results is shown above. Compare this with the calculated values.

Footnotes

[1] This section is based on [10], [14], and [16].

[2] https://research.ibm.com/blog/factor-15-shors-algorithm

[3] https://en.wikipedia.org/wiki/Integer_factorization_records#Records_for_efforts_by_quantum_computers